In [ ]:
from Bio.PDB import *
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

In [ ]:
pdb_list = pd.read_csv("", header=None)
compound_list = pd.read_csv("")
DATA_DIR = ""

In [ ]:
rna_list = pdb_list
sm_list = list(compound_list["id"].values)

In [ ]:
for i in rna_list[0]:
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(i, file_format="pdb", pdir=DATA_DIR)

In [ ]:
parser = PDBParser() 
parser.QUIET = True

In [ ]:
def is_het(residue):
    res = residue.id[0]
    # if res != " " and res != "W":
    #     print(res)
    return res != " " and res != "W" and res.split("_")[1].upper() in sm_list

class ResidueSelect(Select):
    def __init__(self, chain, residue):
        self.chain = chain
        self.residue = residue

    def accept_chain(self, chain):
        return chain.id == self.chain.id

    def accept_residue(self, residue):
        """ Recognition of heteroatoms - Remove water molecules """
        return residue == self.residue and is_het(residue)

class NonHetSelect(Select):
    def accept_residue(self, residue):
        return 1 if residue.id[0] == " " else 0
    
# class ChainSelect(Select):
#     def __init__(self, chain, residue):
#         self.chain = chain
#         self.residue = residue

#     def accept_chain(self, chain):
#         return chain.id == self.chain.id

#     def accept_residue(self, residue):
#         """ Recognition of heteroatoms - Remove water molecules """
#         return residue == self.residue and not is_het(residue)

In [ ]:
def check_ligand(structure, file_name, output_dir):
    io = PDBIO()
    io.set_structure(structure)
    model = structure[0]
    io.set_structure(model)
    # directory = f"{output_dir}/{file_name}"
    # if not os.path.exists(directory):
    #     os.makedirs(directory)
    res_list = []
    ligand_counter = 0
    for chain in model:
        for residue in chain:
            if not is_het(residue):
                continue
            res = residue.id[0]
            code = res.split("_")[1].upper()
            if code not in res_list:
                res_list.append(code)
    return res_list

            
        # break


In [ ]:
map_dict = {}
for i in tqdm(rna_list[0]):
    structure = parser.get_structure(i, f"")
    ligand_code_list = check_ligand(structure, i, "")
    map_dict[i] = ligand_code_list
    # break
    # print(i)

In [ ]:
map_dict

In [ ]:
import pickle 

save_path = ""
with open(save_path, 'wb') as f:
    pickle.dump(map_dict, f)